In [3]:
import pandas as pd

# Step 1: 读取数据
df_sales = pd.read_csv("Sales_Customor_detail.CSV", encoding='latin1')
df_tags = pd.read_csv("customer_behavior_tags.csv")

# 只保留 anomaly_only 客户
anomaly_only_list = df_tags[df_tags['Tag'] == 'anomaly_only']['Name'].tolist()
df_sales = df_sales[df_sales['Name'].isin(anomaly_only_list)]

# 清洗日期与金额
df_sales['Date'] = pd.to_datetime(df_sales['Date'], errors='coerce')
df_sales = df_sales.dropna(subset=['Date'])
df_sales = df_sales[df_sales['Amount'] > 0]

# 提取字段
df_sales = df_sales[['Name', 'Date', 'Item']]
df_sales['YearMonth'] = df_sales['Date'].dt.to_period('M')
latest_month = df_sales['YearMonth'].max()

# 构建客户 × 商品 × 月份表
grouped = df_sales.groupby(['Name', 'Item', 'YearMonth']).size().reset_index(name='order_count')

# 筛选过去常买但最近没买的商品
dropoff_rows = []

for (name, item), sub in grouped.groupby(['Name', 'Item']):
    past_12 = sub[sub['YearMonth'] < latest_month].sort_values('YearMonth')
    
    if past_12['YearMonth'].nunique() >= 2:
        if latest_month not in sub['YearMonth'].values:
            last_purchase = past_12['YearMonth'].max()
            dropoff_rows.append({
                'Name': name,
                'Item': item,
                'last_purchase_month': str(last_purchase),
                'purchase_months': past_12['YearMonth'].nunique()
            })

# 输出中间结果
df_dropoff_anomaly = pd.DataFrame(dropoff_rows)

# ✅ 筛选最后一次购买时间在 2025年2月 到 2025年7月之间的
df_dropoff_anomaly['last_purchase_month'] = pd.to_datetime(df_dropoff_anomaly['last_purchase_month'], errors='coerce')

mask = (df_dropoff_anomaly['last_purchase_month'] >= '2025-02-01') & \
       (df_dropoff_anomaly['last_purchase_month'] <= '2025-07-31')

df_filtered = df_dropoff_anomaly[mask]

# ✅ 导出最终结果
df_filtered.to_csv("dropoff_anomaly_only_filtered.csv", index=False)
df_filtered.head()



,Name,Item,last_purchase_month,purchase_months
0,399 Vietnamese Kitchen,012271 (SOLEX 98MM FLAT STRAW SLOT LID FOR 12-...,2025-05-01,3
1,399 Vietnamese Kitchen,012286 (SOLEX 20 OZ CLEAR PET CUPS 98MM (20/50...,2025-05-01,3
2,399 Vietnamese Kitchen,"01319 (SOLEX LARGE LID FOR 3.25, 4, 5.5 OZ POR...",2025-03-01,3
3,399 Vietnamese Kitchen,01325 (SOLEX 4 OZ CLEAR PP PORTION CUP (10/250...,2025-03-01,3
4,399 Vietnamese Kitchen,"02092 (SOLEX 10x7x12"" PREMIUM NATURAL KRAFT PA...",2025-03-01,7


NameError: name 'df' is not defined